In [1]:
import redis
import time
from datetime import datetime, timedelta

In [2]:
r = redis.Redis(decode_responses=True)

In [17]:
"""
SETRANGE
- Overwrites part of the string stored at key, starting at the specified offset, for the entire length of value.
- Note that the maximum offset that you can set is 2^29 -1 (536870911), as Redis Strings are limited to 512 megabytes. 
- If you need to grow beyond this size, you can use multiple keys.
- TC - O(1), not counting the time taken to copy the new string in place. Usually, this string is very small so the amortized complexity is O(1). Otherwise, complexity is O(M) with M being the length of the value argument.
- Pattern - can be used to change part of key in O(1)
"""
r.flushall()
r.set("key1", "Hello World")
print(r.get("key1")) # Hello World
r.setrange("key1", 6, "Redis")
print(r.get("key1")) # Hello Redis
r.setrange("key1", 2, " - SETRANGE")
print(r.get("key1")) # He - SETRANGE

Hello World
Hello Redis
He - SETRANGE


In [19]:
"""
If the offset is larger than the current length of the string at key, the string is padded with zero-bytes to make offset fit
"""

r.flushall()
r.set("key1", "Hello World")
print(r.get("key1")) # Hello World
r.setrange("key1", 100, " Redis")
print(r.get("key1")) # Hello World Redis

Hello World
Hello World                                                                                          Redis


In [24]:
"""
Non-existing keys are considered as empty strings, so this command will make sure it holds a string large enough to be able to set value at offset
"""
r.flushall()
r.setrange("key1", 1, "Hello World")
print(r.get("key1")) # Hello World
r.flushall()
r.setrange("key1", 100, "Hello World")
print(r.get("key1")) # Hello World

 Hello World
                                                                                                    Hello World


In [ ]:
"""
Warning: When setting the last possible byte and the string value stored at key does not yet hold a string value, 
or holds a small string value, Redis needs to allocate all intermediate memory which can block the server for some time. 
On a 2010 MacBook Pro, setting byte number 536870911 (512MB allocation) takes ~300ms, 
setting byte number 134217728 (128MB allocation) takes ~80ms, setting bit number 33554432 (32MB allocation) takes ~30ms 
and setting bit number 8388608 (8MB allocation) takes ~8ms. 
Note that once this first allocation is done, subsequent calls to SETRANGE for the same key will not have the allocation overhead.
"""